In [1]:
import numpy as np
import pandas as pd
from pandas import read_csv
import pickle
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.preprocessing import OneHotEncoder

In [2]:
df = pd.read_csv('dataset_for_normalization_OneHOtencoding for week 24.csv', parse_dates = ['datetime'], index_col = ['datetime'])
df.head(2)

,week_X-2,week_X-3,week_X-4,MA_X-4,dayOfWeek,weekend,holiday,hourOfDay,T2M_toc,DEMAND
datetime,,,,,,,,,,
2015-01-31 01:00:00,962.2865,906.9580,970.3450,938.004850,1,1,0,1,25.308496,954.2018
2015-01-31 02:00:00,933.3221,863.5135,912.1755,900.284075,1,1,0,2,25.141443,913.8660


In [4]:
df.isnull().sum().sum()

0

In [5]:
dataset = df.iloc[:].values 
dataset.shape

(46968, 10)

In [49]:
df2=pd.read_csv('Test week 24.csv', parse_dates = ['datetime'], index_col = ['datetime'])
df2.drop(['Holiday_ID'], axis = 1, inplace = True)
df2.head(2)

,week_X-2,week_X-3,week_X-4,MA_X-4,dayOfWeek,weekend,holiday,hourOfDay,T2M_toc,DEMAND
datetime,,,,,,,,,,
2020-06-13 01:00:00,1008.5706,987.0052,1042.7384,1020.189200,1,1,1,1,25.864984,1010.6531
2020-06-13 02:00:00,981.9116,957.1975,1018.0739,992.719025,1,1,1,2,25.748865,976.2663


# Function to split data for train, validate and test

In [50]:
def t_v_t(dataset,train_p,validation_p,test_p):
    '''
    \n\nt_v_t(dataset,train_p,validation_p,test_p)\n
    dataset = DataFrame
    train_p = Percentage of Traing Data
    validation_p = Percentage of Validation Data
    test_p = Percentage of Test Data
    \n\nExample:
    to call this Function;
    train_set , validation_set , test_set = t_v_t(df,70,20,10)
    '''
    train_p=train_p/100
    validation_p=validation_p/100
    test_p=test_p/100
    train=int(np.round(len(dataset)*train_p))                                 #70
    validation=int(np.round(len(dataset)*validation_p))                       #20
    test=int(np.round(len(dataset)*test_p))                                   #10
    train_set,validation_set,test_set=dataset[:train],dataset[train:train+validation],dataset[train+validation:len(dataset)]
    return train_set,validation_set,test_set

In [51]:
print(t_v_t.__doc__)


    

t_v_t(dataset,train_p,validation_p,test_p)

    dataset = DataFrame
    train_p = Percentage of Traing Data
    validation_p = Percentage of Validation Data
    test_p = Percentage of Test Data
    

Example:
    to call this Function;
    train_set , validation_set , test_set = t_v_t(df,70,20,10)
    


# Normalization

In [52]:
train_set , validation_set , test_set = t_v_t(df,80,20,0);

In [53]:
test_set=df2

In [54]:
print(train_set.shape)
print(validation_set.shape)
print(test_set.shape)

(37574, 10)
(9394, 10)
(168, 10)


# MinMaxScaler

In [55]:
train_set_load1             = train_set['DEMAND'].values.reshape(-1, 1) 
validation_set_load1        = validation_set['DEMAND'].values.reshape(-1, 1)
test_set_load1              = test_set['DEMAND'].values.reshape(-1, 1) 

scaler = MinMaxScaler(feature_range=(0, 1))
scaler.fit(train_set_load1)

scaled_train_set_load1      = scaler.transform(train_set_load1)
scaled_validation_set_load1 = scaler.transform(validation_set_load1)
scaled_test_set_load1       = scaler.transform(test_set_load1)

In [56]:
pickle.dump(scaler, open ("scaler.pkl",'wb') )

In [57]:
scaled_train_set_load1.shape

(37574, 1)

In [58]:
train_set_load             = train_set[['week_X-2','week_X-3','week_X-4','MA_X-4','T2M_toc']]              
validation_set_load        = validation_set[['week_X-2','week_X-3','week_X-4','MA_X-4','T2M_toc']]
test_set_load              = test_set[['week_X-2','week_X-3','week_X-4','MA_X-4','T2M_toc']]
#.........................................................................
scaler = MinMaxScaler(feature_range=(0, 1))
scaler.fit(train_set_load)
#.........................................................................
scaled_train_set_load      = scaler.transform(train_set_load)
scaled_validation_set_load = scaler.transform(validation_set_load)
scaled_test_set_load       = scaler.transform(test_set_load)

In [59]:
scaled_train_set_load.shape

(37574, 5)

In [60]:
scaled_train_set_load          = np.concatenate((scaled_train_set_load1 , scaled_train_set_load),axis=1) 
scaled_validation_set_load     = np.concatenate((scaled_validation_set_load1 , scaled_validation_set_load),axis=1) 
scaled_test_set_load           = np.concatenate((scaled_test_set_load1 , scaled_test_set_load),axis=1) 

In [61]:
scaled_train_set_load.shape

(37574, 6)

# One Hot Encoding

In [62]:
train_set.columns

Index(['week_X-2', 'week_X-3', 'week_X-4', 'MA_X-4', 'dayOfWeek', 'weekend',
       'holiday', 'hourOfDay', 'T2M_toc', 'DEMAND'],
      dtype='object')

In [63]:
train_setO     = train_set[:].values

weekend        = train_setO[:,5:6]
holiday        = train_setO[:,6:7]


en_weekend     = OneHotEncoder(handle_unknown='ignore')
en_holiday     = OneHotEncoder(handle_unknown='ignore')


weekendf       = en_weekend.fit(weekend)
holidayf       = en_holiday.fit(holiday)

weekendt       = weekendf.transform(weekend).toarray()
holidayt       = holidayf.transform(holiday).toarray()

train_setO     = np.concatenate((weekendt , holidayt, ),axis=1) 
train_setO.shape

(37574, 4)

In [64]:
validation_setO = validation_set[:].values


weekend        = validation_setO[:,5:6]
holiday        = validation_setO[:,6:7]


en_weekend     = OneHotEncoder(handle_unknown='ignore')
en_holiday     = OneHotEncoder(handle_unknown='ignore')


weekendf       = en_weekend.fit(weekend)
holidayf       = en_holiday.fit(holiday)

weekendt       = weekendf.transform(weekend).toarray()
holidayt       = holidayf.transform(holiday).toarray()

val_one_hot = np.concatenate((weekendt , holidayt, ),axis=1)  
val_one_hot.shape

(9394, 4)

In [65]:
test_set.columns

Index(['week_X-2', 'week_X-3', 'week_X-4', 'MA_X-4', 'dayOfWeek', 'weekend',
       'holiday', 'hourOfDay', 'T2M_toc', 'DEMAND'],
      dtype='object')

In [66]:
test_setO      = test_set[:].values


weekend        = test_setO[:,5:6]
holiday        = test_setO[:,6:7]


en_weekend     = OneHotEncoder(handle_unknown='ignore')
en_holiday     = OneHotEncoder(handle_unknown='ignore')


weekendf       = en_weekend.fit(weekend)
holidayf       = en_holiday.fit(holiday)

weekendt       = weekendf.transform(weekend).toarray()
holidayt       = holidayf.transform(holiday).toarray()

test_one_hot = np.concatenate((weekendt , holidayt, ),axis=1)   
test_one_hot.shape 

(168, 4)

In [67]:
train_set.columns

Index(['week_X-2', 'week_X-3', 'week_X-4', 'MA_X-4', 'dayOfWeek', 'weekend',
       'holiday', 'hourOfDay', 'T2M_toc', 'DEMAND'],
      dtype='object')

In [68]:
cyclic_train = train_set[['dayOfWeek','hourOfDay']]
cyclic_train = cyclic_train[:].values

sin_weekt    = np.sin(2*np.pi*cyclic_train[:,0:1]/6)
cos_weekt    = np.cos(2*np.pi*cyclic_train[:,0:1]/6)
  
sin_hourt    = np.sin(2*np.pi*cyclic_train[:,1:2]/24)
cos_hourt    = np.cos(2*np.pi*cyclic_train[:,1:2]/24)

cyclic_train = np.concatenate((sin_weekt, cos_weekt, sin_hourt,cos_hourt), axis=1) 

In [69]:
cyclic_validation = validation_set[['dayOfWeek','hourOfDay']]
cyclic_validation = cyclic_validation[:].values


sin_weekt         = np.sin(2*np.pi*cyclic_validation[:,0:1]/6)
cos_weekt         = np.cos(2*np.pi*cyclic_validation[:,0:1]/6)

sin_hourt         = np.sin(2*np.pi*cyclic_validation[:,1:2]/24)
cos_hourt         = np.cos(2*np.pi*cyclic_validation[:,1:2]/24)

cyclic_validation = np.concatenate((sin_weekt, cos_weekt, sin_hourt,cos_hourt), axis=1)

In [70]:
cyclic_test = test_set[['dayOfWeek','hourOfDay']]
cyclic_test = cyclic_test[:].values


sin_weekt   = np.sin(2*np.pi*cyclic_test[:,0:1]/6)
cos_weekt   = np.cos(2*np.pi*cyclic_test[:,0:1]/6)

sin_hourt   = np.sin(2*np.pi*cyclic_test[:,1:2]/24)
cos_hourt   = np.cos(2*np.pi*cyclic_test[:,1:2]/24)

cyclic_test = np.concatenate((sin_weekt, cos_weekt, sin_hourt,cos_hourt), axis=1)

In [71]:
train = np.concatenate((scaled_train_set_load, train_setO, cyclic_train), axis=1)

In [72]:
validation = np.concatenate((scaled_validation_set_load, val_one_hot, cyclic_validation), axis=1)

In [73]:
test = np.concatenate((scaled_test_set_load, test_one_hot, cyclic_test), axis=1)

In [77]:
train_df = pd.DataFrame(data = train.transpose(), index=['DEMAND','week_X-2',
                                                       'week_X-3','week_X-4',
                                                       'MA_X-4','T2M_toc',
                                                       'weekend1','weekend2',
                                                       'holiday1','holiday2',
                                                       'dayOfWeek1','dayOfWeek2',
                                                       'hourOfDay1','hourOfDay2']).transpose()

In [79]:
train_df.to_csv('train_Normalized for week 24.csv', index=False)

In [80]:
validation_df = pd.DataFrame(data = validation.transpose(), index=['DEMAND','week_X-2',
                                                       'week_X-3','week_X-4',
                                                       'MA_X-4','T2M_toc',
                                                       'weekend1','weekend1',
                                                       'holiday1','holiday2',
                                                       'dayOfWeek1','dayOfWeek2',
                                                       'hourOfDay1','hourOfDay2']).transpose()

In [81]:
validation_df.to_csv('validation_Normalized for week 24.csv', index=False)

In [82]:
test_df = pd.DataFrame(data = test.transpose(), index=['DEMAND','week_X-2',
                                                       'week_X-3','week_X-4',
                                                       'MA_X-4','T2M_toc',
                                                       'weekend1','weekend1',
                                                       'holiday1','holiday2',
                                                       'dayOfWeek1','dayOfWeek2',
                                                       'hourOfDay1','hourOfDay2']).transpose()

In [83]:
test_df.to_csv('test_Normalized for week 24.csv', index=False)

In [84]:
test_df.shape

(168, 14)